import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [32]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc
from pyspark.sql.functions import when

PATH = 'ds_salaries.csv'

create local SparkSession

In [33]:
spark=SparkSession.builder.appName('spark_task_4').getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

read csv with inferschema

In [34]:
df = spark.read.csv(PATH, header=True, inferSchema=True)



read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [35]:
df

DataFrame[_c0: int, work_year: int, experience_level: string, employment_type: string, job_title: string, salary: int, salary_currency: string, salary_in_usd: int, employee_residence: string, remote_ratio: int, company_location: string, company_size: string]

write schema of scv on screen

In [36]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [37]:
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('work_year', StringType(), True),
    StructField('experience_level', StringType(), True),
    StructField('employment_type', StringType(), True),
    StructField('job_title', StringType(), True),
    StructField('salary', IntegerType(), True),
    StructField('salary_currency', StringType(), True),
    StructField('salary_in_usd', IntegerType(), True),
    StructField('employee_residence', StringType(), True),
    StructField('remote_ratio', IntegerType(), True),
    StructField('company_location', StringType(), True),
    StructField('company_size', StringType(), True),
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [38]:
%%time
df_schema = spark.read.option("header",True) \
               .schema(schema) \
               .csv("ds_salaries.csv")

CPU times: user 382 µs, sys: 1.73 ms, total: 2.11 ms
Wall time: 13.7 ms


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [39]:
df_schema.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: string (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [40]:
df_schema.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [41]:
display(df_schema.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [42]:
df_job_title = df_schema.select('job_title').distinct()

print all rows from df_job_titles without truncating jobs

In [43]:
df_job_title.show(df_job_title.count(), truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
|Research S

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [44]:
df_analytic = df_schema.groupBy('job_title') \
        .agg(max('salary_in_usd').alias('max_salary'),
            avg('salary_in_usd').alias('avg_salary'),
            min('salary_in_usd').alias('min_salary')
            )

print all rows from df_analytic without trancating jobs

In [45]:
df_analytic.show(df_analytic.count(), truncate=False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |max_salary|avg_salary        |min_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |5409      |5409.0            |5409      |
|Lead Data Engineer                      |276000    |139724.5          |56000     |
|Head of Machine Learning                |79039     |79039.0           |79039     |
|Data Specialist                         |165000    |165000.0          |165000    |
|Data Analytics Lead                     |405000    |405000.0          |405000    |
|Machine Learning Scientist              |260000    |158412.5          |12000     |
|Lead Data Analyst                       |170000    |92203.0           |19609     |
|Data Engineering Manager                |174000    |123227.2          |59303     |
|Staff Data Scientist                    |105000    |105000.0          |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [46]:
wndw_config = Window.orderBy(desc('avg_salary'))
df_analytic = df_analytic.withColumn('row_id', row_number().over(wndw_config))

print all data from df_analytic

In [47]:
df_analytic.show()

+--------------------+----------+------------------+----------+------+
|           job_title|max_salary|        avg_salary|min_salary|row_id|
+--------------------+----------+------------------+----------+------+
| Data Analytics Lead|    405000|          405000.0|    405000|     1|
|Principal Data En...|    600000| 328333.3333333333|    185000|     2|
|Financial Data An...|    450000|          275000.0|    100000|     3|
|Principal Data Sc...|    416000|215242.42857142858|    148261|     4|
|Director of Data ...|    325000|          195074.0|    130026|     5|
|      Data Architect|    266400| 177873.9090909091|     90700|     6|
|Applied Data Scie...|    380000|          175655.0|     54238|     7|
|  Analytics Engineer|    205300|          175000.0|    135000|     8|
|     Data Specialist|    165000|          165000.0|    165000|     9|
|        Head of Data|    235000|          160162.6|     32974|    10|
|Machine Learning ...|    260000|          158412.5|     12000|    11|
|Data 

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [48]:
df_analytic = df_analytic.select('row_id', 'job_title', 'avg_salary', 'min_salary', 'max_salary')

print df_analytic now

In [49]:
df_analytic.show()

+------+--------------------+------------------+----------+----------+
|row_id|           job_title|        avg_salary|min_salary|max_salary|
+------+--------------------+------------------+----------+----------+
|     1| Data Analytics Lead|          405000.0|    405000|    405000|
|     2|Principal Data En...| 328333.3333333333|    185000|    600000|
|     3|Financial Data An...|          275000.0|    100000|    450000|
|     4|Principal Data Sc...|215242.42857142858|    148261|    416000|
|     5|Director of Data ...|          195074.0|    130026|    325000|
|     6|      Data Architect| 177873.9090909091|     90700|    266400|
|     7|Applied Data Scie...|          175655.0|     54238|    380000|
|     8|  Analytics Engineer|          175000.0|    135000|    205300|
|     9|     Data Specialist|          165000.0|    165000|    165000|
|    10|        Head of Data|          160162.6|     32974|    235000|
|    11|Machine Learning ...|          158412.5|     12000|    260000|
|    1

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [50]:
wndw_config = Window.partitionBy('experience_level')
df_exp_lvl = df_schema.withColumn('biggest_salary', max('salary_in_usd').over(wndw_config))


print here df_exp_lvl

In [51]:
df_exp_lvl.show()

+---+---------+----------------+---------------+--------------------+-------+---------------+-------------+------------------+------------+----------------+------------+--------------+
| id|work_year|experience_level|employment_type|           job_title| salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|biggest_salary|
+---+---------+----------------+---------------+--------------------+-------+---------------+-------------+------------------+------------+----------------+------------+--------------+
|  5|     2020|              EN|             FT|        Data Analyst|  72000|            USD|        72000|                US|         100|              US|           L|        250000|
| 10|     2020|              EN|             FT|      Data Scientist|  45000|            EUR|        51321|                FR|           0|              FR|           S|        250000|
| 12|     2020|              EN|             FT|      Data Scientist|  3500

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [52]:
df_best = df_exp_lvl.filter(col('salary_in_usd') == col('biggest_salary')).select('id', 'experience_level', 'biggest_salary', 'employee_residence')

print df_best

In [53]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 97|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [54]:
df_best = df_best.dropDuplicates(['experience_level'])


print df_best

In [55]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [56]:
df_new_best = df_best.withColumn('experience_level',
                                       when(df_best.experience_level == 'MI', 'middle')
                                      .when(df_best.experience_level == 'SE', 'senior')
                                      .otherwise('Null')).drop('id')

print df_new_best

In [57]:
df_new_best.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|            Null|        250000|                US|
|            Null|        600000|                US|
|          middle|        450000|                US|
|          senior|        412000|                US|
+----------------+--------------+------------------+



write df_new_best like 1.csv and load then it to df_final

In [58]:
df_new_best.write.csv('1/file', header=True)
df_final = spark.read.csv('1/file', header=True, inferSchema=True)

print df_final

In [59]:
df_final.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|            Null|        250000|                US|
|            Null|        600000|                US|
|          middle|        450000|                US|
|          senior|        412000|                US|
+----------------+--------------+------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [60]:
df_final = df_final.filter(df_final.experience_level != 'Null').join(df_schema,
                        (df_final.biggest_salary == df_schema.salary_in_usd) &
                        (df_final.employee_residence == df_schema.employee_residence), 'inner')

print df_final

In [61]:
df_final.show()

+----------------+--------------+------------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|experience_level|biggest_salary|employee_residence| id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+----------------+--------------+------------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|          middle|        450000|                US| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|
|          senior|        412000|                US| 63|     2020|              SE|             FT|      Data Scientist|

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [62]:
the_biggest_salary_usd = df_final.agg({'salary_in_usd':'max'})

In [63]:
the_biggest_salary_usd.show()


+------------------+
|max(salary_in_usd)|
+------------------+
|            450000|
+------------------+



It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system